In [ ]:
import pandas as pd
import io
import zipfile
import requests
from xml.etree.ElementTree import parse
import pandas as pd
import time
import datetime

In [ ]:
#### NEW ####

In [ ]:
!pip install cchardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for cchardet: filename=cchardet-2.1.7-cp310-cp310-linux_x86_64.whl size=289401 sha256=e2821b046cb9c230f7b4e361d86f778f4445c30592ca43fc48c0183cc33d532d
  Stored in directory: /root/.cache/pip/wheels/ee/e0/ab/e01326f15c59438d080b1496dbab8091e952ec72f35e3c437e
Successfully built cchardet


In [ ]:
import requests
import zipfile
import os

url="https://opendart.fss.or.kr/api/corpCode.xml"
api_key="5016e83354f0447e5828f09ecb72a9c373a0bbf4"
params={'crtfc_key' : api_key}

response=requests.get(url, params=params).content

with open ('./id.zip','wb') as fp:
  fp.write(response.content)

zf=zipfile.ZipFile('./id.zip')
zf.extractall()
xml_path=os.path.abspath('./CORPCODE.xml')

In [ ]:
import cchardet
from bs4 import BeautifulSoup
with open(xml_path, 'r',encoding="utf-8") as fp:
  corpcode=BeautifulSoup(fp.read(),'lxml')

<ipython-input-5-f0e3594e9de3>:4: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  corpcode=BeautifulSoup(fp.read(),'lxml')


In [ ]:
import xml.etree.ElementTree as ET

tree=ET.parse(xml_path)
root=tree.getroot()
tags_list=root.findall('list')

In [ ]:
import pandas as pd

def convert(tag:ET.Element) -> dict:
  conv={}
  for child in list(tag):
    conv[child.tag]=child.text
  return conv

tags_list_dict = [convert(x) for x in tags_list]
df=pd.DataFrame(tags_list_dict)

In [ ]:
df #고유번호/정식명칭/종목코드/최종변경일자

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,,20170630
1,00430964,굿앤엘에스,,20170630
2,00388953,크레디피아제이십오차유동화전문회사,,20170630
3,00179984,연방건설산업,,20170630
4,00420143,브룩스피알아이오토메이션잉크,,20170630
...,...,...,...,...
107471,01835069,아로하,,20240510
107472,01839199,호프인터내셔널,,20240510
107473,01823392,포스텍네트웍스,,20240510
107474,01837003,농업회사법인복순도가,,20240510


In [ ]:
df[df['stock_code']!=' '] #종목코드가 있는 곳은 약 4천개뿐(==상장한 기업)

,corp_code,corp_name,stock_code,modify_date
1949,00260985,한빛네트,036720,20170630
1961,00264529,엔플렉스,040130,20170630
1962,00358545,동서정보기술,055000,20170630
2709,00231567,애드모바일,032600,20170630
3785,00359614,리더컴,056140,20170630
...,...,...,...,...
107407,00317104,라이온켐텍,171120,20240401
107408,00651901,에어부산,298690,20240401
107409,00661847,화인베스틸,133820,20240401
107410,01428203,케이씨씨글라스,344820,20240401


In [ ]:
#상장기업들만 !!
stock_df=df[df['stock_code']!=" "]
stock_df=stock_df[['corp_code','corp_name','stock_code']].drop_duplicates()
stock_df

,corp_code,corp_name,stock_code
1949,00260985,한빛네트,036720
1961,00264529,엔플렉스,040130
1962,00358545,동서정보기술,055000
2709,00231567,애드모바일,032600
3785,00359614,리더컴,056140
...,...,...,...
107407,00317104,라이온켐텍,171120
107408,00651901,에어부산,298690
107409,00661847,화인베스틸,133820
107410,01428203,케이씨씨글라스,344820


In [ ]:
def get_items(KEY, CORP_CODE, YEAR, RPT_CODE):
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
  from lxml import html
  from urllib.request import Request, urlopen
  from urllib.parse import urlencode, quote_plus, unquote

  url='http://opendart.fss.or.kr/api/fnlttSingIAcnt.xml'
  params={'crtfc_key': KEY
          , 'corp_code': CORP_CODE
          , 'bsns_year' : YEAR
          , 'report_code' : RPT_CODE}

  response=requests.get(url, params=params).content.decode('UTF-8')

  xml_obj=BeautifulSoup(response, 'html.parser')
  rows=xml_obj.findAll("list")
  return rows

In [ ]:
import urllib3
import warnings

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
r = requests.get(url, params=params, verify=False)
warnings.resetwarnings()

In [ ]:
! pip install pyOpenSSL

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
KEY="5016e83354f0447e5828f09ecb72a9c373a0bbf4"
YEAR="2023"
RPT_CODE="11011"

corp_list=stock_df['corp_code'].to_list()

item_list=['bsns_year','stock_code','report_code','fs_div','sj_div','account_nm','thstrm_nm','thstrm_dt','thstrm_amount','thstrm_add_amount','frmtrm_nm','frmtrm_df','frmtrm_amount','frmtrm_add_amount','bfefrmtrm_nm','bfefrmtrm_dt','bfefrmtrm_amount','currency']
#item_list=['사업연도','종목코드','보고서코드','연결/개별구분','재무제표구분','계정명','당기명','당기일자','당기금액','당기누적금액','전기명','전기일자','전기금액','전기누적금액','전전기명','전전기일자','전전기금액','통화']

fs_list=[]
for c in corp_list:
  CORP_CODE=str(c).zfill(8)
  items=get_items(KEY,CORP_CODE, YEAR, RPT_CODE)

  for i in range(0, len(items)):
    fs_item_list =[]
    for item in item_list:
      try:
        value=items[i].find(item).text
      except:
        value=''

      fs_item_list.append(value)

    fs_list.append(fs_item_list)
  print(CORP_CODE)

00260985
00264529
00358545
00231567
00359614
00153551
00344746
00261188
00268020
00269287
00475286
00250775
00114321
00191108
00167378
00225210
00223346
00174129
00114826
00173333
00687599
00256292
00271431
00154727
00246189
00154204
00152631
00358095
00510215
00347619
00372013
00353735
00186540
00128759
00535588
00264981
00231521
00159908
00259493
00359641
00161639
00211271
00388272
00264796
00134909
00236368
00684547
00251385
00262682
00234555
00507257
00417963
00543523
00300566
00299376
00112068
00257194
00477372
00465410
00331760
00200761
00268297
00161301
00237260
00297970
00107358
00267474
00134051
00596066
00347363
00150226
00152914
00158839
00159281
00210537
00240370
00247887
00268358
00316488
00264130
00540304
00617299
00262275
00200798
00335924
00111591
00189343
00252588
00267997
00299686
00463050
00238913
00107446
00114303
00318617
00267775
00394574
00110176
00136554
00144562
00370547
00242439
00134060
00392600
00259855
00385451
00258944
00798530
00451268
00336570
00363325
0

SSLError: HTTPSConnectionPool(host='opendart.fss.or.kr', port=443): Max retries exceeded with url: /error1.html (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))

In [ ]:
#### NEW2 ####

In [1]:
import requests
from io import BytesIO
from zipfile import ZipFile
from xml.etree.ElementTree import parse
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [2]:
url="https://opendart.fss.or.kr/api/corpCode.xml"
KEY="5016e83354f0447e5828f09ecb72a9c373a0bbf4"
params={'crtfc_key' : KEY}

In [3]:
response=requests.get(url, params=params).content

with ZipFile(BytesIO(response)) as zipfile:
  zipfile.extractall('corpCode')

In [5]:
xmlTree = parse('/content/corpCode/CORPCODE.xml')
root=xmlTree.getroot()
raw_list = root.findall('list')

In [6]:
corp_list=[]

for l in range(0,len(raw_list)):
  corp_code=raw_list[l].findtext('corp_code')
  corp_name=raw_list[l].findtext('corp_name')
  stock_code=raw_list[l].findtext('stock_code')
  modify_date=raw_list[l].findtext('modify_date')

  corp_list.append([corp_code, corp_name, stock_code, modify_date])
  print(l)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
102480
102481
102482
102483
102484
102485
102486
102487
102488
102489
102490
102491
102492
102493
102494
102495
102496
102497
102498
102499
102500
102501
102502
102503
102504
102505
102506
102507
102508
102509
102510
102511
102512
102513
102514
102515
102516
102517
102518
102519
102520
102521
102522
102523
102524
102525
102526
102527
102528
102529
102530
102531
102532
102533
102534
102535
102536
102537
102538
102539
102540
102541
102542
102543
102544
102545
102546
102547
102548
102549
102550
102551
102552
102553
102554
102555
102556
102557
102558
102559
102560
102561
102562
102563
102564
102565
102566
102567
102568
102569
102570
102571
102572
102573
102574
102575
102576
102577
102578
102579
102580
102581
102582
102583
102584
102585
102586
102587
102588
102589
102590
102591
102592
102593
102594
102595
102596
102597
102598
102599
102600
102601
102602
102603
102604
102605
102606
102607
102608
102609
102610
102611
102612
102613
102614
102615
102616
10261

In [7]:
from pandas import DataFrame
from datetime import datetime

corp_df=DataFrame(corp_list, columns=['고유번호','정식명칭','종목코드','최종변경일자'])
corp_df.head()

,고유번호,정식명칭,종목코드,최종변경일자
0,00434003,다코,,20170630
1,00430964,굿앤엘에스,,20170630
2,00388953,크레디피아제이십오차유동화전문회사,,20170630
3,00179984,연방건설산업,,20170630
4,00420143,브룩스피알아이오토메이션잉크,,20170630


In [9]:
import pandas as pd

stock_df=corp_df[corp_df['종목코드']!= ' ']

stock_df.head()

,고유번호,정식명칭,종목코드,최종변경일자
1949,00260985,한빛네트,036720,20170630
1961,00264529,엔플렉스,040130,20170630
1962,00358545,동서정보기술,055000,20170630
2709,00231567,애드모바일,032600,20170630
3785,00359614,리더컴,056140,20170630


In [10]:
def get_items(KEY, CORP_CODE, YEAR, RPT_CODE):
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
  from lxml import html
  from urllib.request import Request, urlopen
  from urllib.parse import urlencode, quote_plus, unquote

  url='http://opendart.fss.or.kr/api/fnlttSingIAcnt.xml'
  params={'crtfc_key': KEY
          , 'corp_code': CORP_CODE
          , 'bsns_year' : YEAR
          , 'report_code' : RPT_CODE}

  response=requests.get(url, params=params).content.decode('UTF-8')

  xml_obj=BeautifulSoup(response, 'html.parser')
  rows=xml_obj.findAll("list")
  return rows

In [ ]:
KEY="5016e83354f0447e5828f09ecb72a9c373a0bbf4"
YEAR="2023"
RPT_CODE="11011"

corp_list=stock_df['고유번호'].to_list()

item_list=['bsns_year','stock_code','report_code','fs_div','sj_div','account_nm','thstrm_nm','thstrm_dt','thstrm_amount','thstrm_add_amount','frmtrm_nm','frmtrm_df','frmtrm_amount','frmtrm_add_amount','bfefrmtrm_nm','bfefrmtrm_dt','bfefrmtrm_amount','currency']
#item_list=['사업연도','종목코드','보고서코드','연결/개별구분','재무제표구분','계정명','당기명','당기일자','당기금액','당기누적금액','전기명','전기일자','전기금액','전기누적금액','전전기명','전전기일자','전전기금액','통화']

fs_list=[]
for c in corp_list:
  CORP_CODE=str(c).zfill(8)
  items=get_items(KEY,CORP_CODE, YEAR, RPT_CODE)

  for i in range(0, len(items)):
    fs_item_list =[]
    for item in item_list:
      try:
        value=items[i].find(item).text
      except:
        value=''

      fs_item_list.append(value)

    fs_list.append(fs_item_list)
  print(CORP_CODE)

00260985
00264529
00358545
00231567
00359614
00153551
00344746
00261188
00268020
00269287
00475286
00250775
00114321
00191108
00167378
00225210
00223346
00174129
00114826
00173333
00687599
00256292
00271431
00154727
00246189
00154204
00152631
00358095
00510215
00347619
00372013
00353735
00186540
00128759
00535588
00264981
00231521
00159908
00259493
00359641
00161639
00211271
00388272
00264796
00134909
00236368
00684547
00251385
00262682
00234555
00507257
00417963
00543523
00300566
00299376
00112068
00257194
00477372
00465410
00331760
00200761
00268297
00161301
00237260
00297970
00107358
00267474
00134051
00596066
00347363
00150226
00152914
00158839
00159281
00210537
00240370
00247887
00268358
00316488
00264130
00540304
00617299
00262275
00200798
00335924
00111591
00189343
00252588
00267997
00299686
00463050
00238913
00107446
00114303
00318617
00267775
00394574
00110176
00136554
00144562
00370547
00242439
00134060
00392600
00259855
00385451
00258944
00798530
00451268
00336570
00363325
0

SSLError: HTTPSConnectionPool(host='opendart.fss.or.kr', port=443): Max retries exceeded with url: /api/fnlttSingIAcnt.xml?crtfc_key=5016e83354f0447e5828f09ecb72a9c373a0bbf4&corp_code=00124054&bsns_year=2023&report_code=11011 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))

In [ ]:
import pandas as pd
from pandas import DataFrame
from datetime import datetime

fs_df=DataFrame(fs_list, columns=[
    '사업연도', '종목코드','보고서코드','연결/개별구분','재무제표구분','계정명','당기명','당기일자','당기금액','당기누적금액','전기명','전기일자','전기금액','전기누적금액','전전기명','전전기일자','전전기금액','통화'])

fs_df.head(5)

In [ ]:
stock_df[stock_df['고유번호']=='00221320']

,고유번호,정식명칭,종목코드,최종변경일자
9052,00221320,에이스디지텍,036550,20170630
